# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.



In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys_file import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# extract the csv file 
output_data_file = "../starter_code/weather_data_final.csv"
output_data_file

# read the csv file 
weather_df = pd.read_csv(output_data_file)
weather_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Richards Bay,-28.78,32.04,70.50,87,93,4.18,ZA,1606087303
1,Khatanga,71.97,102.50,14.92,96,100,20.38,RU,1606087379
2,Bredasdorp,-34.53,20.04,60.80,87,26,3.36,ZA,1606087229
3,Korla,41.76,86.15,18.10,80,40,3.47,CN,1606087382
4,Tasiilaq,65.61,-37.64,21.20,85,46,2.24,GL,1606087383


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# creating a heat map

# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity_1 = weather_df["Humidity"].astype(float)

In [4]:
# Plot Heatmap
# fig = gmaps.figure()
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_1, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

In [5]:
# plt.savefig('vacation-humidity-heatmap.png')

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to fit my ideal type of weather condition. My criteria is as follow:
# A max temperature lower or equal to 85 degrees and higher than 50 degrees.
# Wind speed < 15
# cloudiness <= 10


# created a new data frame that has these for criterias 
my_types_df = pd.DataFrame(weather_df, columns = ["City", "Country", "Max Temp", "Wind Speed", "Cloudiness", "Humidity", "Lat", "Lng"])
my_types_df

# setting up my conditions 
my_ideal_type_df = my_types_df.loc[(my_types_df["Wind Speed"] < 15) & (my_types_df["Cloudiness"] <= 10) & \
                                   (my_types_df["Max Temp"] <= 85) & (my_types_df["Max Temp"] > 50)].dropna()

my_ideal_type_df

# plt.savefig('ideal-weather.png')


,City,Country,Max Temp,Wind Speed,Cloudiness,Humidity,Lat,Lng
10,Makkah al Mukarramah,SA,72.32,4.79,0,74,21.43,39.83
15,Carnarvon,AU,73.40,11.41,0,78,-24.87,113.63
17,Busselton,AU,66.99,7.16,0,67,-33.65,115.33
19,Bluff,NZ,57.00,5.01,3,66,-46.60,168.33
35,Chui,UY,60.48,8.48,0,79,-33.70,-53.46
...,...,...,...,...,...,...,...,...
555,Panabá,MX,72.91,7.20,6,92,21.28,-88.27
563,Kontagora,NG,73.13,4.47,6,37,10.40,5.47
580,Clarence Town,AU,80.01,5.82,0,73,-32.58,151.78
589,Ambilobe,MG,73.38,4.18,6,80,-13.20,49.05


In [7]:
len(my_ideal_type_df)

78

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Create a hotel_df
hotel_df = my_ideal_type_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
10,Makkah al Mukarramah,SA,21.43,39.83,
15,Carnarvon,AU,-24.87,113.63,
17,Busselton,AU,-33.65,115.33,
19,Bluff,NZ,-46.60,168.33,
35,Chui,UY,-33.70,-53.46,


In [9]:
# using google API to find the hotel nearby my ideal cities

google_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
# use iterrows to iterate through pandas dataframe

for index, row in hotel_df.iterrows():
    # extract the city name, latitude, longtitude from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dictionary
    params["location"] = f"{lat},{lng}"
    
 # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(google_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")



Retrieving Results for Index 10: Makkah al Mukarramah.
Closest hotel in Makkah al Mukarramah is Hotel Fairmont Makkah Clock Royal Tower.
------------
Retrieving Results for Index 15: Carnarvon.
Closest hotel in Carnarvon is The Gascoyne Hotel.
------------
Retrieving Results for Index 17: Busselton.
Closest hotel in Busselton is Abbey Beach Resort.
------------
Retrieving Results for Index 19: Bluff.
Closest hotel in Bluff is Land's End Boutique Hotel.
------------
Retrieving Results for Index 35: Chui.
Closest hotel in Chui is Bertelli Chuí Hotel.
------------
Retrieving Results for Index 37: Maceió.
Closest hotel in Maceió is Jatiúca Hotel & Resort.
------------
Retrieving Results for Index 41: Malakal.
Closest hotel in Malakal is Nile Palace Hotel.
------------
Retrieving Results for Index 42: Caçu.
Closest hotel in Caçu is Império Hotel.
------------
Retrieving Results for Index 48: Carballo.
Closest hotel in Carballo is Hotel Pensua Punta Del Este.
------------
Retrieving Results 

Retrieving Results for Index 594: Korba.
Closest hotel in Korba is Blue Diamond The Hotel.
------------
-------End of Search-------


In [11]:
# save the photo of all the hotel names
# plt.savefig('vacation-nearest-hotels.png')

In [12]:
# Display the dataframe after located hotels 
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
10,Makkah al Mukarramah,SA,21.43,39.83,Hotel Fairmont Makkah Clock Royal Tower
15,Carnarvon,AU,-24.87,113.63,The Gascoyne Hotel
17,Busselton,AU,-33.65,115.33,Abbey Beach Resort
19,Bluff,NZ,-46.60,168.33,Land's End Boutique Hotel
35,Chui,UY,-33.70,-53.46,Bertelli Chuí Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))

In [15]:
# save the photo 
# plt.savefig('ideal-cities-hotels.png')
